In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from bs4 import BeautifulSoup
import urllib
import codecs
import time
import random

In [2]:
censorship_phrase_utf8 = '根据相关法律法规和政策'
censorship_phrase_decoded = codecs.encode(censorship_phrase_utf8.decode('utf8'), 'unicode_escape')

def has_censorship(keyword):
    no_results_phrase_utf8 = '抱歉，未找到“%s”相关结果。' % keyword
    no_results_phrase_decoded = codecs.encode(no_results_phrase_utf8.decode('utf8'), 'unicode_escape')
    
    r = urllib.urlopen('http://s.weibo.com/weibo/%s&Refer=index' % keyword).read()
    time.sleep(random.randint(1, 3))
    
    if censorship_phrase_decoded in r:
        return True
    elif no_results_phrase_decoded in r:
        return None
    else:
        return False
    #soup = BeautifulSoup(r, 'html.parser')

In [3]:
test_keywords = ['hello',
                'lxb',
                '习胞子']

In [7]:
for k in test_keywords:
    print k, "\t", has_censorship(k)

 hello 	False
lxb 	True
习胞子 	None
